In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from collections import deque
import random

In [3]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from collections import deque

# Define hyperparameters
BATCH_SIZE = 32
GAMMA = 0.95
EPSILON = 1.0
EPSILON_DECAY = 0.995
EPSILON_MIN = 0.01
MEMORY_SIZE = 10000
LEARNING_RATE = 0.001
TRAINING_EPISODES = 500

# Load data
data = pd.read_csv('grocery_data3.csv')
print(data)
# Define state size and action size
state_size = len(data.columns) - 1
action_size = len(data['product_id'].unique())
print(data)

items = pd.DataFrame({
    'product_id': data['product_id'],
    'product_name': data['product_name'],
    'category': data['category'],
    'price': data['price'],
    'brand': data['brand'],
    'shelf_life': data['shelf_life'],
    'usual_storage': data['usual_storage']
})

      user_id  product_id  quantity                      timestamp  age  \
0         421          48         2  2022-01-01 00:00:00.000000000   45   
1         162          48         1  2022-10-27 19:41:21.008100808   23   
2         892          48         3  2022-11-07 04:41:24.248424840   74   
3         887          48         3  2022-03-26 00:28:13.609360936   31   
4         838          48         1  2022-01-04 15:22:07.452745274   65   
...       ...         ...       ...                            ...  ...   
9995      521          30         5  2022-04-01 21:35:50.495049505   52   
9996      265          30         4  2022-10-08 05:38:43.312331232   54   
9997      338          30         1  2022-06-01 21:01:25.328532852   42   
9998      441          30         1  2022-06-03 10:35:32.133213320   51   
9999      846          30         2  2022-04-25 03:53:00.918091808   42   

      gender    location product_name category     price    brand  shelf_life  \
0       male   New

In [6]:


# Load the data from the CSV file
data = pd.read_csv('grocery_data3.csv')

# Define the state space
state_cols = ['user_id', 'age', 'gender', 'location', 'day_of_week', 'time_of_day', 'weather']

# Define the action space
action_cols = ['product_id', 'quantity']

# Define the reward function
def reward_function(row):
    # The reward function could be based on various factors such as profit, sales, etc.
    # Here's an example reward function based on sales:
    price = row['price']
    quantity = row['quantity']
    return price * quantity

# Define the function to encode the state
def encode_state(state):
    # One-hot encode the categorical variables in the state
    encoded_state = pd.get_dummies(state, columns=['gender', 'location', 'day_of_week', 'time_of_day', 'weather'])
    return encoded_state

# Define the function to decode the state
def decode_state(encoded_state):
    # Decode the one-hot encoded categorical variables in the state
    state = encoded_state[state_cols]
    gender_cols = [col for col in encoded_state.columns if col.startswith('gender_')]
    state['gender'] = np.argmax(encoded_state[gender_cols].values, axis=1)
    location_cols = [col for col in encoded_state.columns if col.startswith('location_')]
    state['location'] = np.argmax(encoded_state[location_cols].values, axis=1)
    day_of_week_cols = [col for col in encoded_state.columns if col.startswith('day_of_week_')]
    state['day_of_week'] = np.argmax(encoded_state[day_of_week_cols].values, axis=1)
    time_of_day_cols = [col for col in encoded_state.columns if col.startswith('time_of_day_')]
    state['time_of_day'] = np.argmax(encoded_state[time_of_day_cols].values, axis=1)
    weather_cols = [col for col in encoded_state.columns if col.startswith('weather_')]
    state['weather'] = np.argmax(encoded_state[weather_cols].values, axis=1)
    return state

# Preprocess data
user_ids = data['user_id'].unique()
num_users = len(user_ids)
user_id_to_index = {user_id: i for i, user_id in enumerate(user_ids)}
item_ids = data['product_id'].unique()
num_items = len(item_ids)
item_id_to_index = {item_id: i for i, item_id in enumerate(item_ids)}

# Split data into training and testing sets
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

# Convert data to sequences
def convert_to_sequences(data):
    sequences = []
    for user_id in data['user_id'].unique():
        user_data = data[data['user_id'] == user_id].sort_values(by='timestamp')
        sequence = []
        for i, row in user_data.iterrows():
            state = np.array([user_id_to_index[user_id], item_id_to_index[row['product_id']], row['quantity'], row['price']])
            action = item_id_to_index[row['product_id']]
            reward = row['price']
            next_state = np.array([user_id_to_index[user_id], item_id_to_index[row['product_id']], row['quantity'], row['price']])
            done = False
            sequence.append((state, action, reward, next_state, done))
        sequences.append(sequence)
    return sequences

train_sequences = convert_to_sequences(train_data)
test_sequences = convert_to_sequences(test_data)




In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import gym

# Define the DQN model
import random
from collections import deque

learning_rate = 0.001
discount_factor = 0.95
batch_size = 32
memory_size = 100000
target_update_freq = 1000
max_steps_per_episode = 100
num_episodes = 500

class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = zip(*batch)
        return state_batch, action_batch, reward_batch, next_state_batch, done_batch

    def __len__(self):
        return len(self.memory)
import numpy as np

class ExplorationPolicy:
    def __init__(self, eps_start, eps_end, eps_decay):
        self.eps_start = eps_start
        self.eps_end = eps_end
        self.eps_decay = eps_decay
    
    def get_eps(self, episode):
        eps = self.eps_end + (self.eps_start - self.eps_end) * math.exp(-1. * episode / self.eps_decay)
        return eps
    
    def select_action(self, agent, state):
        eps = self.get_eps(agent.episode)
        if random.random() < eps:
            return random.choice(range(agent.action_dim))
        else:
            q_values = agent.model.predict(state)
            return np.argmax(q_values[0])

class DQNAgent:
    def __init__(self, env, model, replay_memory, exploration_policy, batch_size, num_episodes, max_steps_per_episode, update_target_network_every):
        self.env = env
        self.replay_memory = replay_memory
        self.exploration_policy = exploration_policy
        self.batch_size = batch_size
        self.num_episodes = num_episodes
        self.max_steps_per_episode = max_steps_per_episode
        self.update_target_network_every = update_target_network_every
        self.model = create_model(env.observation_space.shape[0], env.action_space.n)
        self.target_model = create_model(env.observation_space.shape[0], env.action_space.n)
        self.target_model.set_weights(self.model.get_weights())
        self.gamma = 0.99
        self.episode = 0 
        self.action_dim = 4
        
    def build_model(self):
        model = Sequential([
            Dense(24, input_shape=(self.env.observation_space.shape[0],), activation='relu'),
            Dense(24, activation='relu'),
            Dense(self.env.action_space.n, activation='linear')
        ])
        model.compile(optimizer=Adam(lr=0.001), loss='mse')
        return model
    def train(self):
        for episode in range(self.num_episodes):
            state = self.env.reset()
            print(state[0].shape)
            print("Shape of state array before reshaping:", state[0].shape)
            state = np.reshape(state[0], [1, self.model.input_shape[1]])
            done = False
            total_reward = 0
            for step in range(self.max_steps_per_episode):
                action = self.exploration_policy.select_action(self, state)
                obj = self.env.step(action)
                next_state=obj[0]
                reward=obj[1]
                done=obj[2]
                info=obj[3]
                next_state = np.reshape(next_state, [1, self.model.input_shape[1]])
                self.replay_memory.push(state, action, reward, next_state, done)
             
                state = next_state
                total_reward += reward
                if done:
                    break
                    
            if len(self.replay_memory) > self.batch_size:
                minibatch = self.replay_memory.sample(self.batch_size)
                self.model.train(minibatch)
            if episode % self.update_target_network_every == 0:
                self.model.update_target_network()
            self.exploration_policy.eps = self.exploration_policy.get_eps(episode+1)
            self.model.episode = episode+1
            self.episode += 1
            print("Episode {}/{}: Total reward = {}, Epsilon = {}".format(episode+1, self.num_episodes, total_reward, self.exploration_policy.eps))
            
        
       from keras.models import Sequential
from keras.layers import Dense

def create_model(state_dim, num_actions):
    model = Sequential()
    model.add(Dense(32, input_dim=state_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_actions, activation='linear'))
    model.compile(loss='mse', optimizer='adam')
    return model

class EpsilonGreedyPolicy:
    def __init__(self, initial_epsilon, min_epsilon, decay_rate):
        self.epsilon = initial_epsilon
        self.min_epsilon = min_epsilon
        self.decay_rate = decay_rate
        
    def get_action(self, model, state):
        if np.random.rand() < self.epsilon:
            # Take a random action
            action = np.random.randint(model.num_actions)
        else:
            # Choose the action with highest Q-value
            q_values = model(state)
            action = np.argmax(q_values)

        # Decay epsilon
        self.epsilon = max(self.min_epsilon, self.epsilon * self.decay_rate)
        
        return action

 

In [ ]:

    
class DQN(tf.keras.Model):
    def __init__(self, env, state_dim, num_actions, replay_memory, exploration_policy, batch_size, num_episodes, max_steps_per_episode, update_target_network_every):
        super(DQN, self).__init__()
        self.env = env
        self.state_dim = state_dim
        self.num_actions = num_actions
        self.replay_memory = replay_memory
        self.exploration_policy = exploration_policy
        self.batch_size = batch_size
        self.num_episodes = num_episodes
        self.max_steps_per_episode = max_steps_per_episode
        self.update_target_network_every = update_target_network_every
        self.model = create_model(state_dim, num_actions)
        self.target_model = create_model(state_dim, num_actions)
        self.target_model.set_weights(self.model.get_weights())

    def update_model(self, batch):
        states = batch["states"]
        actions = batch["actions"]
        rewards = batch["rewards"]
        next_states = batch["next_states"]
        dones = batch["dones"]

        # Compute the target Q values using the target network
        target_q_values = self.target_model(next_states)
        max_target_q_values = tf.reduce_max(target_q_values, axis=-1)
        target_q_values = rewards + (1 - dones) * self.discount_factor * max_target_q_values

        # Compute the predicted Q values using the main network
        with tf.GradientTape() as tape:
            q_values = self.model(states)
            action_masks = tf.one_hot(actions, depth=self.num_actions)
            predicted_q_values = tf.reduce_sum(q_values * action_masks, axis=-1)
            loss = tf.reduce_mean(tf.square(target_q_values - predicted_q_values))

        # Compute the gradients of the loss with respect to the model parameters
        gradients = tape.gradient(loss, self.model.trainable_variables)

        # Apply the gradients to update the model weights
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

        

    # Define the train function
    def train(self):
        for episode in range(self.num_episodes):
            state = self.env.reset()
#             print("type")
            state = np.reshape(state, [1, self.state_dim])
            done = False
            total_reward = 0

            for step in range(self.max_steps_per_episode):
                # Epsilon-greedy exploration
                action = self.exploration_policy.get_action(self.model, state)

                # Take a step
                next_state, reward, done, info = self.env.step(action)
                next_state = np.reshape(next_state, [1, self.state_dim])
                total_reward += reward

                # Save the transition to replay memory
                self.replay_memory.add_transition(state, action, reward, next_state, done)

                # Sample a batch of transitions from replay memory
                batch = self.replay_memory.sample_batch(self.batch_size)

                # Update the DQN model
                self.update_model(batch)

                # Update the target network every few episodes
                if (episode + 1) % self.update_target_network_every == 0:
                    self.target_model.set_weights(self.model.get_weights())

                if done:
                    break

                state = next_state

            print("Episode {}: Total reward = {}".format(episode + 1, total_reward))
    
    
    def call(self, state):
        x = self.hidden_layer_1(state)
        x = self.hidden_layer_2(x)
        q_values = self.output_layer(x)
        return q_values


In [ ]:

# Create the environment
env = gym.make('CartPole-v1')
# env=GroceryEnv()
# print("grocery env",env)
# Initialize the replay memory
replay_memory = ReplayMemory(capacity=10000)
# print("replay memeory",replay_memory)
# Define the exploration policy
exploration_policy = ExplorationPolicy(1.0, 0.01, 0.995)

# state_dim = env.observation_space.shape

state_dim = env.observation_space.shape[0]
# print("action space",len(env.action_space),env.action_space )
# action_dim = env.action_space.n if isinstance(env.action_space, gym.spaces.Discrete) else env.action_space.shape[0][0]

# if isinstance(env.action_space, spaces.Discrete):
#     action_dim = env.action_space.n
# else:
#     action_dim = tuple(s.n for s in env.action_space)



state_dim = env.observation_space.shape[0]
# print("space dim",)
# print("state dimm ",state_dim)
# print(len(env.action_space) )
action_dim = env.action_space.n
hidden_dim = 128
learning_rate = 0.001
discount_factor = 0.99
# Define the training parameters
batch_size = 32
num_episodes = 1000
max_steps_per_episode = 100
update_target_network_every = 500

# Initialize the DQN model
# model = DQN(state_dim, action_dim, hidden_dim, learning_rate, discount_factor)
# Initialize the DQN model
model = DQN(env, state_dim, action_dim, replay_memory, exploration_policy, batch_size, num_episodes, max_steps_per_episode, update_target_network_every)

#######

# Train the DQN model
dqn_agent = DQNAgent(env, model, replay_memory, exploration_policy, batch_size, num_episodes, max_steps_per_episode, update_target_network_every)
print(dqn_agent)
dqn_agent.train()
 

In [15]:
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# # NCF
# # Load data
# data = pd.read_csv('grocery_data3.csv')

# # Convert data to user-item matrix
# user_item_matrix = pd.pivot_table(data, values='quantity', index='user_id', columns='product_id', fill_value=0)

# # Define hyperparameters
# NUM_USERS = len(user_item_matrix)
# NUM_ITEMS = len(user_item_matrix.columns)
# EMBEDDING_SIZE = 32
# BATCH_SIZE = 64
# EPOCHS = 10
# LEARNING_RATE = 0.001

# # Split data into training and validation sets
# train_size = int(0.8 * len(data))
# train_data = data.iloc[:train_size]
# val_data = data.iloc[train_size:]

# # Define neural network model
# input_user = tf.keras.layers.Input(shape=(1,))
# embedding_user = tf.keras.layers.Embedding(input_dim=NUM_USERS, output_dim=EMBEDDING_SIZE)(input_user)
# flatten_user = tf.keras.layers.Flatten()(embedding_user)

# input_item = tf.keras.layers.Input(shape=(1,))
# embedding_item = tf.keras.layers.Embedding(input_dim=NUM_ITEMS, output_dim=EMBEDDING_SIZE)(input_item)
# flatten_item = tf.keras.layers.Flatten()(embedding_item)

# dot_product = tf.keras.layers.Dot(axes=1)([flatten_user, flatten_item])
# model = tf.keras.Model(inputs=[input_user, input_item], outputs=dot_product)

# # Compile model
# model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE), loss='mean_squared_error')

# # Define function to generate batches of training data
# def generate_batches(data):
#     num_batches = len(data) // BATCH_SIZE
#     for i in range(num_batches):
#         batch_data = data.iloc[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
#         user_ids = np.array(batch_data['user_id'])
#         item_ids = np.array(batch_data['product_id'])
#         ratings = np.array(batch_data['quantity'])
#         yield [user_ids, item_ids], ratings

# # Train model
# for epoch in range(EPOCHS):
#     for batch in generate_batches(train_data):
#         inputs, targets = batch
#         model.train_on_batch(inputs, targets)
#     # Evaluate model on validation set
#     val_loss = model.evaluate([val_data['user_id'], val_data['product_id']], val_data['quantity'], verbose=0)
#     print(f"Epoch {epoch+1}/{EPOCHS}: Validation loss = {val_loss:.4f}")

# # Recommend items to a user
# def recommend_items(user_id, top_n=5):
#     user_ratings = user_item_matrix.loc[user_id].values
#     user_ratings = user_ratings.reshape(1, -1)
#     item_indices = np.argsort(-user_ratings)[0][:top_n]
#     item_ids = [user_item_matrix.columns[i] for i in item_indices]
#     item_ratings = user_ratings[0, item_indices]
#     return item_ids, item_ratings


Epoch 1/10: Validation loss = 11.1647
Epoch 2/10: Validation loss = 11.1651
Epoch 3/10: Validation loss = 11.1656
Epoch 4/10: Validation loss = 11.1659
Epoch 5/10: Validation loss = 11.1654
Epoch 6/10: Validation loss = 11.1635
Epoch 7/10: Validation loss = 11.1600
Epoch 8/10: Validation loss = 11.1552
Epoch 9/10: Validation loss = 11.1498
Epoch 10/10: Validation loss = 11.1441


In [17]:
# item_ids, item_ratings=recommend_items(1,5)
# print(item_ids, item_ratings)

[85, 54, 3, 7, 37] [5. 5. 5. 5. 4.]


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from collections import deque
import random

# Load data
data = pd.read_csv('grocery_data3.csv')

# Convert data to user-item matrix
user_item_matrix = pd.pivot_table(data, values='quantity', index='user_id', columns='product_id', fill_value=0)

# Define hyperparameters
NUM_USERS = len(user_item_matrix)
NUM_ITEMS = len(user_item_matrix.columns)
EMBEDDING_SIZE = 32
BATCH_SIZE = 64
EPISODES = 100
EPSILON = 1.0
EPSILON_DECAY = 0.999
EPSILON_MIN = 0.01
GAMMA = 0.99
LEARNING_RATE = 0.001
MEMORY_SIZE = 10000

# Split data into training and validation sets
train_size = int(0.8 * len(data))
train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:]

# Define neural network model
input_state = tf.keras.layers.Input(shape=(NUM_ITEMS,))
dense1 = tf.keras.layers.Dense(128, activation='relu')(input_state)
dense2 = tf.keras.layers.Dense(64, activation='relu')(dense1)
output = tf.keras.layers.Dense(NUM_ITEMS, activation='linear')(dense2)
model = tf.keras.Model(inputs=input_state, outputs=output)

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE), loss='mean_squared_error')

# Define replay buffer
memory = deque(maxlen=MEMORY_SIZE)






# Define function to select an action
def select_action(state, epsilon):
    if np.random.rand() < epsilon:
        return random.randrange(NUM_ITEMS)
    else:
        return np.argmax(model.predict(state))

# Define function to train the model
def train_model():
    if len(memory) < BATCH_SIZE:
        return
    batch = random.sample(memory, BATCH_SIZE)
    states = np.zeros((BATCH_SIZE, NUM_ITEMS))
    next_states = np.zeros((BATCH_SIZE, NUM_ITEMS))
    actions, rewards, done = [], [], []
    for i in range(BATCH_SIZE):
        states[i] = batch[i][0]
        actions.append(batch[i][1])
        rewards.append(batch[i][2])
        next_states[i] = batch[i][3]
        done.append(batch[i][4])
    targets = model.predict(states)
    next_q_values = model.predict(next_states)
    for i in range(BATCH_SIZE):
        if done[i]:
            targets[i][actions[i]] = rewards[i]
        else:
            targets[i][actions[i]] = rewards[i] + GAMMA * np.max(next_q_values[i])
    model.fit(states, targets, epochs=1, verbose=0)
def recommend_items(user_id, n=5):
    user_items = user_item_matrix.loc[user_id].values.reshape(1, -1)
    item_scores = model.predict(user_items)[0]
    item_indices = item_scores.argsort()[-n:][::-1]
    recommended_items = []
    for i in item_indices:
        if item_scores[i] > 0:
            recommended_items.append(items[i])
    return recommended_items
# Train model
for episode in range(EPISODES):
    state = user_item_matrix.loc[random.choice(user_item_matrix.index)].values.reshape(1, -1)
    total_reward = 0
    epsilon = max(EPSILON_MIN, EPSILON_DECAY * EPSILON)
    for t in range(NUM_ITEMS):
        action = select_action(state, epsilon)
        reward = user_item_matrix.iloc[:, action].sum()
        next_state = np.copy(state)
        next_state[:, action] = reward
        done = t == NUM_ITEMS - 1
        memory.append((state, action, reward, next_state, done))
        total_reward += reward
        state = next_state
        train_model()
        if done:
            break
    print(f"Episode {episode+1}/{EPISODES}: Total reward =  {total_reward}")
    if (episode+1) % 10 == 0:
        print("Recommended items for user 1:", recommend_items(1))


2/2 [==============================] - 0s 0s/step
Episode 1/100: Total reward =  29166.666666666668
2/2 [==============================] - 0s 2ms/step
Episode 2/100: Total reward =  29551.5
2/2 [==============================] - 0s 2ms/step
Episode 3/100: Total reward =  28667.0
2/2 [==============================] - 0s 0s/step
Episode 4/100: Total reward =  28745.0
2/2 [==============================] - 0s 0s/step
Episode 5/100: Total reward =  28416.833333333332
2/2 [==============================] - 0s 3ms/step
Episode 6/100: Total reward =  29385.833333333332
2/2 [==============================] - 0s 3ms/step
Episode 7/100: Total reward =  28945.5
2/2 [==============================] - 0s 3ms/step
Episode 8/100: Total reward =  29187.833333333332
2/2 [==============================] - 0s 2ms/step
Episode 9/100: Total reward =  28712.0
2/2 [==============================] - 0s 2ms/step
Episode 10/100: Total reward =  28821.000000000004
1/1 [==============================] - 0s 24ms/

KeyError: 40

In [18]:
model.save('recommendation_DQA.h5')
print("Model saved successfully!")


Model saved successfully!


In [19]:
recommended_items=recommend_items(1,5)
print(recommended_items)

([85, 54, 3, 7, 37], array([5., 5., 5., 5., 4.]))
